This file contains steps for aggregating many raster files (.tiff) into a single panda-friendly file (.csv).

It uses the Global Rainfed Crop-Production raster files from SPAM as an example.

In [1]:
import numpy as np
import pandas as pd
from os import listdir, system
from os.path import isfile, join, isdir
import georasters as gr

Convert all Crop Production Rasters into CSVs

## Method 1: Using GDAL

In [12]:
# Does the convertion.
def convert_tif_to_csv_gdal(source, target):
    system('gdal_translate -of xyz -co ADD_HEADER_LINE=YES -co COLUMN_SEPARATOR="," ' + source + ' ' + target)

In [13]:
#*_TA	all technologies together, ie complete crop
#*_TI	irrigated portion of crop
#*_TH	rainfed high inputs portion of crop
#*_TL	rainfed low inputs portion of crop
#*_TS	rainfed subsistence portion of crop
#*_TR	rainfed portion of crop (= TA - TI, or TH + TL + TS)

crop_portion = 'TR'

In [14]:
# crop_files_path: path to directory containing all raster files we want to convert.
# target_dir_path: path to directory where we want to store all converted csvs.

crop_files_path = '../data/crop-production/geotiff_global_prod/'
target_dir_path = './my_spam_csvs'

In [15]:
files = [f for f in listdir(crop_files_path) if '.tif' in f and '.xml' not in f and '_' + crop_portion + '_' in f]

for file in files:
    convert_tif_to_csv(crop_files_path+file, target_dir_path + '/' + file.split('_')[4]+'.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/crop-production/geotiff_global_prod/'

After the above is done running you should have a target directory filled with '.csv' files

Now we'll combine all the tables into one.

In [ ]:
# Get all converted files
csvs = [f for f in listdir(target_dir_path) if '.csv' in f]

In [ ]:
# First we'll get a base dataframe to merge all others into. We'll just pick the first file.
base = pd.read_csv(target_dir_path + '/' + csvs[0])
# And substitute the generic column name by the crop name.
base = base.rename(columns={'Z': csvs[0].split('.')[0]})

In [ ]:
for csv in csvs[1:]:
    df_crop = pd.read_csv(target_dir_path + '/' + csv)
    crop_name = csv.split('.')[0]
    df_crop = df_crop.rename(columns={'Z': crop_name})
    # Join tables
    base = pd.concat([base, df_crop[crop_name]], axis=1)

In [ ]:
base.head()

In [ ]:
# For some reason the tiff to csv translation adds a bunch of -3.402823466385289e+38.
base.replace(-3.402823466385289e+38, 0.0, inplace=True)

In [ ]:
base.head()

In [ ]:
# Write result table to a new csv file.
base.to_csv('./rain_fed_production.csv', encoding='utf-8', index=False)

In [ ]:
# Now enter this in a file you'd like to load the data into.
# df = pd.read_csv('./rain_fed_production.csv')

### Climate Data

In [40]:
files_path = '../data/climate/csvs'

# Get all converted files
csvs = [f for f in listdir(files_path) if '.csv' in f]
# First we'll get a base dataframe to merge all others into. We'll just pick the first file.
base = pd.read_csv(files_path + '/' + csvs[0])
# And substitute the generic column name by the crop name.
base = base.rename(columns={'Z': csvs[0].split('_')[3].split('.')[0]})

In [41]:
for csv in csvs[1:]:
    df_var = pd.read_csv(files_path + '/' + csv)
    var_name = csv.split('_')[3].split('.')[0]
    df_var = df_var.rename(columns={'Z': var_name})
    # Join tables
    base = pd.concat([base, df_var[var_name]], axis=1)

In [27]:
# For some reason the tiff to csv translation adds a bunch of -3.402823466385289e+38.
base.replace(-np.inf, 0.0, inplace=True)

In [42]:
# Reorder
base = base[['X', 'Y', '01', '02', '03','04', '05','06', '07','08', '09','10', '11','12', '13','14', '15', '16', '17', '18']]

In [46]:
# base

In [44]:
base.to_csv('./world_clim_bio_vars.csv', encoding='utf-8', index=False)

## Method 2: Using GeoRasters

In [8]:
def convert_tif_to_csv_georasters(source, target):
    # Get first crop to be base dataframe.
    data = gr.from_file(source)
    df = data.to_pandas()
    df.drop(columns=['row', 'col'])
    df = df[['x', 'y', 'value']]
    df.to_csv(target, encoding='utf-8', index=False)

In [9]:
convert_tif_to_csv_georasters('/Volumes/Christian_2015/project-eden/original-data/SPAM/geotiff_global_prod/SPAM2005V3r2_global_P_TS_YAMS_S.tif', './converted_test.csv')

In [18]:
#*_TA	all technologies together, ie complete crop
#*_TI	irrigated portion of crop
#*_TH	rainfed high inputs portion of crop
#*_TL	rainfed low inputs portion of crop
#*_TS	rainfed subsistence portion of crop
#*_TR	rainfed portion of crop (= TA - TI, or TH + TL + TS)

crop_portion = 'TR'

In [19]:
# crop_files_path: path to directory containing all raster files we want to convert.
# target_dir_path: path to directory where we want to store all converted csvs.

crop_files_path = '/Volumes/Christian_2015/project-eden/original-data/SPAM/geotiff_global_prod/'
target_dir_path = '/Volumes/Christian_2015/project-eden/converted_csvs/my_spam_csvs_georasters/'

In [20]:
files = [f for f in listdir(crop_files_path) if '.tif' in f and '.xml' not in f and '_' + crop_portion + '_' in f]

for file in files:
    convert_tif_to_csv_georasters(crop_files_path+file, target_dir_path + '/' + file.split('_')[4]+'.csv')

### Climate Data

In [ ]:
climate_files_path

In [26]:
climate_files_path = '/Volumes/Christian_2015/project-eden/original-data/WorldClim/wc2.0_5m_bio/'
target_dir_path = '/Volumes/Christian_2015/project-eden/converted_csvs/my_worldclim_csvs_georasters/'

files = [f for f in listdir(climate_files_path) if '.tif' in f]

for file in files:
    convert_tif_to_csv_georasters(climate_files_path+file, target_dir_path + file.split('_')[3].split('.')[0]+'.csv')

## Aggregating tables into one

In [53]:
def aggregate_same_source_csvs(source_dir_path):
    # Get all converted files
    csvs = [f for f in listdir(source_dir_path) if '.csv' in f]
    # First we'll get a base dataframe to merge all others into. We'll just pick the first file.
    base = pd.read_csv(source_dir_path + csvs[0])
    # And substitute the generic column name by the crop name.
    base = base.rename(columns={'value': csvs[0].split('.')[0].lower()})
    
    # Now the rest.
    for csv in csvs[1:]:
        df = pd.read_csv(source_dir_path + csv)
        var_name = csv.split('.')[0].lower()
        df = df.rename(columns={'value': var_name})
        # Join tables
        base = pd.concat([base, df[var_name]], axis=1)
    
    return base

### Crop Data

In [45]:
source_dir_path = '/Volumes/Christian_2015/project-eden/converted_csvs/my_spam_csvs_georasters/'
target_path = '/Volumes/Christian_2015/project-eden/my_aggregates/spam_production_georasters.csv'

df_spam_aggr = aggregate_same_source_csvs(source_dir_path)

In [48]:
# Export
df_spam_aggr.to_csv(target_path, encoding='utf-8', index=False)

### Climata Data

In [54]:
source_dir_path = '/Volumes/Christian_2015/project-eden/converted_csvs/my_worldclim_csvs_georasters/'
target_path = '/Volumes/Christian_2015/project-eden/my_aggregates/worldclim_biovars_georasters.csv'

df_worldclim = aggregate_same_source_csvs(source_dir_path)

In [58]:
# Export
df_worldclim.to_csv(target_path, encoding='utf-8', index=False)

## Merging crop and climate tables

In [59]:
df_clim = pd.read_csv('/Volumes/Christian_2015/project-eden/my_aggregates/worldclim_biovars_georasters.csv')
df_spam = pd.read_csv('/Volumes/Christian_2015/project-eden/my_aggregates/spam_production_georasters.csv')

In [72]:
len(df_clim['x'])

3170214

In [73]:
len(df_spam['x'])

747682

In [92]:
def merge_tables(df_1, df_2):
    # Remove row-conflict by rounding X/Y coordinates to the nearest 8 decimal points.
    df_1['x'] = round(df_1['x'], 8)
    df_1['y'] = round(df_1['y'], 8)

    df_2['x'] = round(df_2['x'], 8)
    df_2['y'] = round(df_2['y'], 8)
    
    # Want to merge the smallest table into the largest.
    if (len(df_1['x']) > len(df_2['x'])):
        larger = df_1
        smaller = df_2
    else:
        larger = df_2
        smaller = df_1
        
  ## Commented out stuff is what you do for tables created with GDAL.
#     larger.set_index(['x', 'y'], inplace=True)
#     smaller.set_index(['x', 'y'], inplace=True)

#     for column in smaller.columns.difference(['x', 'y']):
#         # Each column of the smaller table will be a new column in the larger table. If
#         larger = pd.concat([larger, smaller[column]], axis=1)
    
    return pd.merge(larger, smaller,  how='left')

In [88]:
merged = merge_tables(df_spam, df_clim)

In [90]:
# Proof that it worked?
# merged
merged.loc[merged['bana'].nlargest(1).index]

x          y         01        02         03          04  \
1714021  78.25  10.416667  27.363042  9.169583  74.139598  207.849957   

                05         06         07         08  ...   sugc  sunf  swpo  \
1714021  35.313999  19.151001  16.162998  26.395167  ...    0.0   0.0   7.4   

         teas  temf  toba  trof   vege  whea  yams  
1714021   0.0   0.0   0.0   1.4  298.2   0.0   0.0  

[1 rows x 63 columns]

In [64]:
# Length should be the same as before
len(merged['x'])

3170214

In [91]:
# Export
merged.to_csv('/Volumes/Christian_2015/project-eden/my_aggregates/production_clim_merged_georasters.csv', encoding='utf-8', index=False)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])